In [ ]:
import glob, os
import numpy as np
import scipy.misc
from PIL import Image
from shutil import copyfile

In [ ]:
# Resize

size = 128, 128
total = 161260

for i in range(33820, total):
    path = "S:\\Projects\\MVC\\images\\" + str(i) + ".jpg"
    im = Image.open(path).resize(size)
    im.save("S:\\Projects\\MVC\\128x128\\" + str(i) + ".jpg", "JPEG")

In [ ]:
# Copy

src_path = "S:\\Projects\\MVC\\128x128_clean\\"
dst_path = "H:\\vincentw\\Projects\\CycleGAN-tensorflow\\datasets\\front2left\\"

for i in range(0, 26844, 4):
    copyfile(src_path + str(i) + ".jpg", dst_path + "trainA\\" + str(i) + ".jpg")
    copyfile(src_path + str(i + 1) + ".jpg", dst_path + "trainB\\" + str(i + 1) + ".jpg")

In [ ]:
# Concat

src_path = "S:\\Projects\\MVC\\128x128_clean\\"
dst_path = "S:\\Projects\\MVC\\128x128_concat\\"

for i in range(0, 26844, 4):
    
    path1 = src_path + str(i) + ".jpg"
    path2 = src_path + str(i+1) + ".jpg"
    im1 = np.array(Image.open(path1))
    im2 = np.array(Image.open(path2))
    im = np.concatenate((im1, im2), axis=1)
    scipy.misc.imsave(dst_path + str(i) + ".jpg", im)

In [ ]:
# Test

path = "S:\\Projects\\MVC\\64x64_test\\"

for file in os.listdir(path):
    im = np.array(Image.open(path + file))
    # A->B
    # im[:,int(im.shape[1]/2):,:] = 255
    # B->A
    im = np.concatenate((im[:,int(im.shape[1]/2):,:], np.full((im.shape[0], int(im.shape[1]/2), im.shape[2]), 255)), axis=1)
    scipy.misc.imsave(path + "_" + file, im)